In [13]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [15]:
subject = 'Bahia - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [16]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Bahia - Produção de Cimento (t)', axis=1)
data

,Ano,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - Consumo de Cimento (t)
0,2008,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,0.783758,0.685154,8.102744,0.620781,0.659869,...,2.868108,11.760983,19.314465,1008.202617,1303.749515,26.386473,23.042487,77.493797,1.586819,2678.337
1,2009,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,0.785624,0.687239,8.129293,0.624755,0.660519,...,2.305885,9.472585,20.202505,1007.958668,1414.729195,25.968454,22.350784,78.022515,1.421020,2825.491
2,2010,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,0.786978,0.689024,8.095385,0.628561,0.660850,...,1.958246,9.332485,20.155807,1008.053425,1424.927462,25.603573,21.719583,78.179391,1.422041,3349.109
3,2011,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,0.787829,0.690520,8.084857,0.632198,0.660869,...,2.079088,10.985689,20.048279,1008.445156,1444.966819,25.564945,21.470218,77.933380,1.514544,3615.893
4,2012,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,0.788133,0.691525,13.555307,0.635421,0.660403,...,2.183282,8.159014,20.344237,1008.651747,1446.757048,25.595626,21.547865,78.542856,1.555217,3744.322
5,2013,1.523819e+08,8.603518e+06,8.785884,1.258886e+08,0.787978,0.692653,14.373236,0.638956,0.659936,...,2.403445,7.913561,20.252405,1008.935896,1429.407124,25.586976,21.584976,78.969598,1.570158,3932.905
6,2014,1.543977e+08,8.508098e+06,8.799997,1.261078e+08,0.787476,0.693707,15.102184,0.642702,0.659347,...,2.748397,10.350442,17.807272,1009.223255,1433.335843,25.757283,21.442184,77.696073,1.560276,4055.140
7,2015,1.558374e+08,8.390263e+06,8.806885,1.261743e+08,0.786515,0.693682,15.644545,0.644677,0.657960,...,2.501529,12.454398,17.456441,1009.318844,1452.882864,25.895216,21.251973,76.266812,1.538323,3732.552
8,2016,1.565535e+08,8.301516e+06,8.804784,1.260566e+08,0.796009,0.701870,15.852241,0.655705,0.662422,...,2.073464,13.111908,19.032574,1009.311974,1455.457303,25.838885,21.111445,75.883665,1.523445,3424.968
9,2017,1.566919e+08,8.249496e+06,8.807487,1.259606e+08,0.795010,0.701585,15.631769,0.655678,0.662461,...,2.177386,9.479812,20.787217,1009.248599,1454.646572,25.882215,21.182739,76.030495,1.479634,3063.300


In [17]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,0.783758,0.685154,8.102744,0.620781,0.659869,5.483335e+08,...,-7723.574973,2.868108,11.760983,19.314465,1008.202617,1303.749515,26.386473,23.042487,77.493797,1.586819
1,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,0.785624,0.687239,8.129293,0.624755,0.660519,5.302764e+08,...,-8961.781709,2.305885,9.472585,20.202505,1007.958668,1414.729195,25.968454,22.350784,78.022515,1.421020
2,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,0.786978,0.689024,8.095385,0.628561,0.660850,5.046425e+08,...,-9895.682385,1.958246,9.332485,20.155807,1008.053425,1424.927462,25.603573,21.719583,78.179391,1.422041
3,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,0.787829,0.690520,8.084857,0.632198,0.660869,4.854622e+08,...,-9989.170714,2.079088,10.985689,20.048279,1008.445156,1444.966819,25.564945,21.470218,77.933380,1.514544
4,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,0.788133,0.691525,13.555307,0.635421,0.660403,4.867661e+08,...,-9429.326328,2.183282,8.159014,20.344237,1008.651747,1446.757048,25.595626,21.547865,78.542856,1.555217
5,1.523819e+08,8.603518e+06,8.785884,1.258886e+08,0.787978,0.692653,14.373236,0.638956,0.659936,5.225848e+08,...,-7157.441850,2.403445,7.913561,20.252405,1008.935896,1429.407124,25.586976,21.584976,78.969598,1.570158
6,1.543977e+08,8.508098e+06,8.799997,1.261078e+08,0.787476,0.693707,15.102184,0.642702,0.659347,5.676413e+08,...,-3453.461327,2.748397,10.350442,17.807272,1009.223255,1433.335843,25.757283,21.442184,77.696073,1.560276
7,1.558374e+08,8.390263e+06,8.806885,1.261743e+08,0.786515,0.693682,15.644545,0.644677,0.657960,5.179392e+08,...,-1442.919387,2.501529,12.454398,17.456441,1009.318844,1452.882864,25.895216,21.251973,76.266812,1.538323
8,1.565535e+08,8.301516e+06,8.804784,1.260566e+08,0.796009,0.701870,15.852241,0.655705,0.662422,4.524144e+08,...,-3128.377228,2.073464,13.111908,19.032574,1009.311974,1455.457303,25.838885,21.111445,75.883665,1.523445
9,1.566919e+08,8.249496e+06,8.807487,1.259606e+08,0.795010,0.701585,15.631769,0.655678,0.662461,4.213689e+08,...,-5386.337108,2.177386,9.479812,20.787217,1009.248599,1454.646572,25.882215,21.182739,76.030495,1.479634


In [18]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2825.491
1     3349.109
2     3615.893
3     3744.322
4     3932.905
5     4055.140
6     3732.552
7     3424.968
8     3063.300
9     2990.152
10    3245.585
11    3803.900
12    3862.535
13    3948.984
14         NaN
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [19]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.801595,0.936957,-1.987321,-1.867929,-1.460829,-1.587856,-1.391013,-1.635715,-0.598881,1.204752,...,-0.569731,2.026352,0.984445,-0.494083,-1.206958,-3.089913,2.478541,2.695570,0.052337,1.398792
1,-1.650542,1.089552,-1.802089,-1.807175,-0.960333,-1.212869,-1.383171,-1.312206,-0.056326,0.820657,...,-0.838980,0.085558,-0.051710,0.005518,-1.706936,-0.276913,0.615792,1.366860,0.603496,-1.378567
2,-1.108275,1.207298,-0.956366,-1.029617,-0.597406,-0.891692,-1.393187,-1.002422,0.220138,0.275392,...,-1.042057,-1.114490,-0.115145,-0.020754,-1.512730,-0.018418,-1.010165,0.154369,0.767031,-1.361464
3,-0.609433,1.026498,-0.274462,-0.421068,-0.369250,-0.622546,-1.396297,-0.706399,0.236503,-0.132596,...,-1.062386,-0.697344,0.633403,-0.081248,-0.709872,0.489519,-1.182293,-0.324643,0.510578,0.188100
4,-0.141312,0.776880,0.103974,0.174367,-0.287683,-0.441660,0.219728,-0.444119,-0.153228,-0.104860,...,-0.940648,-0.337667,-0.646476,0.085254,-0.286461,0.534896,-1.045577,-0.175489,1.145924,0.869421
5,0.275387,0.494258,0.363149,0.670072,-0.329395,-0.238888,0.461353,-0.156393,-0.542565,0.657046,...,-0.446625,0.422337,-0.757614,0.033591,0.295906,0.095128,-1.084122,-0.104202,1.590780,1.119708
6,0.605385,0.015214,0.701796,0.926438,-0.463954,-0.049203,0.676692,0.148528,-1.033902,1.615453,...,0.358807,1.613108,0.345772,-1.342011,0.884853,0.194709,-0.325214,-0.378496,0.263198,0.954156
7,0.841077,-0.576356,0.867096,1.004133,-0.721442,-0.053598,0.836911,0.309292,-2.192189,0.558230,...,0.796000,0.760923,1.298413,-1.539385,1.080764,0.690167,0.289436,-0.743877,-1.226730,0.586418
8,0.958309,-1.021897,0.816661,0.866585,1.824027,1.419319,0.898266,1.206866,1.532636,-0.835563,...,0.429496,-0.716758,1.596124,-0.652671,1.066684,0.755421,0.038417,-1.013821,-1.626140,0.337201
9,0.980971,-1.283056,0.881520,0.754314,1.556137,1.368124,0.833136,1.204684,1.564684,-1.495939,...,-0.061498,-0.358019,-0.048438,0.334470,0.936796,0.734872,0.231503,-0.876871,-1.473078,-0.396703


In [20]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2825.491
1     3349.109
2     3615.893
3     3744.322
4     3932.905
5     4055.140
6     3732.552
7     3424.968
8     3063.300
9     2990.152
10    3245.585
11    3803.900
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [21]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.416125,-1.086745,-0.129344,-0.207609,0.000408,0.737609,0.749286,0.972824,-0.47095,1.108405,...,0.964765,-1.482779,-2.260447,1.940009,0.112434,-1.032368,-0.187335,-0.1229,0.783824,-1.669544


In [22]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    3862.535
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [23]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [24]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [26]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[114815337, 2886642491, 565208747, 1198518202, 333346173, 291010760, 1139955526, 2343417258, 2834916197, 769577146, 1973559810, 1580616078, 1480536575, 3445321051, 683297678, 4134735000, 2307957438, 4006587862, 3269896613, 2986916015, 2833143403, 3903385731, 3596871491, 3710511800, 636585396, 1652881293, 1997550969, 78213935, 2663022065, 1414107386, 2188992751, 4184288539, 2779593108, 4061120805, 3361540630, 2259807423, 1796323669, 2099023434, 4180589956, 25169659, 3795441185, 2255076476, 3992638196, 3119874297, 205347227, 701322991, 1963901052, 383621326, 420035625, 621777433, 1986896344, 286089630, 1475811827, 137180252, 2235851965, 1986283319, 2354677770, 4282962472, 2164826976, 3699911930, 179831671, 3607818859, 4098867455, 2758534934, 1058664979, 3947801812, 2682400267, 2305814370, 819232621, 4076565112, 2481023122, 1350933801, 3332885076, 660811730, 3790960746, 3592165945, 2080924719, 975033360, 388102586, 2772449603, 3766292444, 434384873, 2707791454, 4213208218, 541875202, 3766

2023-09-14 13:56:10.365749: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 13:56:10.987512: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 13:56:11.019530: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 177829.8125


Step: 57 ___________________________________________
val_loss: 132302.703125


Step: 58 ___________________________________________
val_loss: 203120.109375


Step: 59 ___________________________________________
val_loss: 198007.625


Step: 60 ___________________________________________
val_loss: 151469.375


Step: 61 ___________________________________________
val_loss: 173505.40625


Step: 62 ___________________________________________
val_loss: 154455.140625


Step: 63 ___________________________________________
val_loss: 132000.40625


Step: 64 ___________________________________________
val_loss: 168520.46875


Step: 65 ___________________________________________
val_loss: 164622.765625


Step: 66 ___________________________________________
val_loss: 146901.515625


Step: 67 ___________________________________________
val_loss: 157104.625


Step: 68 ___________________________________________
val_loss: 187079.0625


Step: 69 _________________________________

In [27]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 903ms/step - loss: 12266084.0000 - val_loss: 13956034.0000
Epoch 2/10000
1/1 [==============================] - 0s 16ms/step - loss: 11940717.0000 - val_loss: 25323654.0000
Epoch 3/10000
1/1 [==============================] - 0s 16ms/step - loss: 10814689.0000 - val_loss: 8132127.0000
Epoch 4/10000
1/1 [==============================] - 0s 18ms/step - loss: 11337104.0000 - val_loss: 15163668.0000
Epoch 5/10000
1/1 [==============================] - 0s 16ms/step - loss: 11539853.0000 - val_loss: 21935282.0000
Epoch 6/10000
1/1 [==============================] - 0s 16ms/step - loss: 11422260.0000 - val_loss: 17595222.0000
Epoch 7/10000
1/1 [==============================] - 0s 16ms/step - loss: 11037293.0000 - val_loss: 14043187.0000
Epoch 8/10000
1/1 [==============================] - 0s 16ms/step - loss: 10597911.0000 - val_loss: 13806843.0000
Epoch 9/10000
1/1 [==============================] - 0s 19ms/step - loss: 9926230.0000 -

1/1 [==============================] - 0s 16ms/step - loss: 2745101.7500 - val_loss: 2388115.2500
Epoch 74/10000
1/1 [==============================] - 0s 16ms/step - loss: 2651955.2500 - val_loss: 3907730.7500
Epoch 75/10000
1/1 [==============================] - 0s 17ms/step - loss: 2940832.7500 - val_loss: 2892603.7500
Epoch 76/10000
1/1 [==============================] - 0s 17ms/step - loss: 2740154.2500 - val_loss: 3283348.5000
Epoch 77/10000
1/1 [==============================] - 0s 16ms/step - loss: 2758683.7500 - val_loss: 3280134.7500
Epoch 78/10000
1/1 [==============================] - 0s 19ms/step - loss: 2637683.5000 - val_loss: 3146823.0000
Epoch 79/10000
1/1 [==============================] - 0s 18ms/step - loss: 2552817.7500 - val_loss: 2286737.5000
Epoch 80/10000
1/1 [==============================] - 0s 17ms/step - loss: 2422516.7500 - val_loss: 2751934.2500
Epoch 81/10000
1/1 [==============================] - 0s 17ms/step - loss: 2477216.7500 - val_loss: 2423253.250

1/1 [==============================] - 0s 18ms/step - loss: 714082.6875 - val_loss: 1729523.6250
Epoch 146/10000
1/1 [==============================] - 0s 19ms/step - loss: 745126.0625 - val_loss: 1733473.7500
Epoch 147/10000
1/1 [==============================] - 0s 19ms/step - loss: 823715.0000 - val_loss: 1678437.8750
Epoch 148/10000
1/1 [==============================] - 0s 19ms/step - loss: 834006.7500 - val_loss: 1445810.1250
Epoch 149/10000
1/1 [==============================] - 0s 19ms/step - loss: 729920.6875 - val_loss: 1936790.0000
Epoch 150/10000
1/1 [==============================] - 0s 23ms/step - loss: 669129.4375 - val_loss: 1451840.2500
Epoch 151/10000
1/1 [==============================] - 0s 19ms/step - loss: 714422.1875 - val_loss: 1598697.8750
Epoch 152/10000
1/1 [==============================] - 0s 18ms/step - loss: 578543.1875 - val_loss: 1495954.7500
Epoch 153/10000
1/1 [==============================] - 0s 22ms/step - loss: 709233.5000 - val_loss: 1501934.6250

1/1 [==============================] - 0s 17ms/step - loss: 274797.0625 - val_loss: 409576.8750
Epoch 219/10000
1/1 [==============================] - 0s 17ms/step - loss: 297994.3438 - val_loss: 255888.5156
Epoch 220/10000
1/1 [==============================] - 0s 18ms/step - loss: 392603.1250 - val_loss: 525986.5000
Epoch 221/10000
1/1 [==============================] - 0s 17ms/step - loss: 298280.5000 - val_loss: 295946.3750
Epoch 222/10000
1/1 [==============================] - 0s 17ms/step - loss: 347148.7812 - val_loss: 458310.8125
Epoch 223/10000
1/1 [==============================] - 0s 19ms/step - loss: 335667.5312 - val_loss: 223912.8750
Epoch 224/10000
1/1 [==============================] - 0s 17ms/step - loss: 286537.7188 - val_loss: 398798.0938
Epoch 225/10000
1/1 [==============================] - 0s 18ms/step - loss: 223733.4219 - val_loss: 366499.2812
Epoch 226/10000
1/1 [==============================] - 0s 16ms/step - loss: 261193.2031 - val_loss: 377220.8750
Epoch 22

1/1 [==============================] - 0s 17ms/step - loss: 201239.7969 - val_loss: 412291.9062
Epoch 292/10000
1/1 [==============================] - 0s 19ms/step - loss: 202601.3125 - val_loss: 318868.3750
Epoch 293/10000
1/1 [==============================] - 0s 18ms/step - loss: 164080.2656 - val_loss: 397183.0625
Epoch 294/10000
1/1 [==============================] - 0s 18ms/step - loss: 195197.6406 - val_loss: 319585.4062
Epoch 295/10000
1/1 [==============================] - 0s 18ms/step - loss: 176235.4531 - val_loss: 371365.8750
Epoch 296/10000
1/1 [==============================] - 0s 18ms/step - loss: 151596.6875 - val_loss: 418380.6250
Epoch 297/10000
1/1 [==============================] - 0s 18ms/step - loss: 217498.4531 - val_loss: 261260.0000
Epoch 298/10000
1/1 [==============================] - 0s 18ms/step - loss: 194631.7500 - val_loss: 505971.0312
Epoch 299/10000
1/1 [==============================] - 0s 17ms/step - loss: 176462.5000 - val_loss: 401208.4375
Epoch 30

...

1/1 [==============================] - 0s 17ms/step - loss: 127545.0234 - val_loss: 228143.1250
Epoch 730/10000
1/1 [==============================] - 0s 18ms/step - loss: 126768.6797 - val_loss: 247989.6562
Epoch 731/10000
1/1 [==============================] - 0s 17ms/step - loss: 113086.1250 - val_loss: 264420.9375
Epoch 732/10000
1/1 [==============================] - 0s 18ms/step - loss: 125740.7578 - val_loss: 249730.0469
Epoch 733/10000
1/1 [==============================] - 0s 17ms/step - loss: 134011.6250 - val_loss: 262369.8125
Epoch 734/10000
1/1 [==============================] - 0s 18ms/step - loss: 125994.5703 - val_loss: 250695.3125
Epoch 735/10000
1/1 [==============================] - 0s 18ms/step - loss: 140657.7500 - val_loss: 256233.9375
Epoch 736/10000
1/1 [==============================] - 0s 17ms/step - loss: 127453.1875 - val_loss: 245501.6250
Epoch 737/10000
1/1 [==============================] - 0s 18ms/step - loss: 126506.5625 - val_loss: 240335.2500
Epoch 73

1/1 [==============================] - 0s 16ms/step - loss: 121923.2188 - val_loss: 180986.4688
Epoch 803/10000
1/1 [==============================] - 0s 17ms/step - loss: 127317.5000 - val_loss: 176006.0625
Epoch 804/10000
1/1 [==============================] - 0s 16ms/step - loss: 125758.0234 - val_loss: 176062.8125
Epoch 805/10000
1/1 [==============================] - 0s 17ms/step - loss: 129386.0391 - val_loss: 173222.6094
Epoch 806/10000
1/1 [==============================] - 0s 16ms/step - loss: 124526.0000 - val_loss: 180626.2500
Epoch 807/10000
1/1 [==============================] - 0s 16ms/step - loss: 125537.7891 - val_loss: 178860.4375
Epoch 808/10000
1/1 [==============================] - 0s 17ms/step - loss: 124200.5000 - val_loss: 182553.6250
Epoch 809/10000
1/1 [==============================] - 0s 16ms/step - loss: 123520.0312 - val_loss: 182378.2031
Epoch 810/10000
1/1 [==============================] - 0s 16ms/step - loss: 115317.8125 - val_loss: 201159.0312
Epoch 81

1/1 [==============================] - 0s 18ms/step - loss: 119896.7188 - val_loss: 223175.7344
Epoch 876/10000
1/1 [==============================] - 0s 17ms/step - loss: 123261.3750 - val_loss: 195775.7812
Epoch 877/10000
1/1 [==============================] - 0s 17ms/step - loss: 120982.1953 - val_loss: 213749.5469
Epoch 878/10000
1/1 [==============================] - 0s 21ms/step - loss: 115032.7812 - val_loss: 222738.8281
Epoch 879/10000
1/1 [==============================] - 0s 17ms/step - loss: 129235.2891 - val_loss: 207031.0000
Epoch 880/10000
1/1 [==============================] - 0s 17ms/step - loss: 127390.6797 - val_loss: 211392.5156
Epoch 881/10000
1/1 [==============================] - 0s 18ms/step - loss: 116726.1875 - val_loss: 212434.1250
Epoch 882/10000
1/1 [==============================] - 0s 19ms/step - loss: 120523.3125 - val_loss: 218010.7969
Epoch 883/10000
1/1 [==============================] - 0s 18ms/step - loss: 117545.7266 - val_loss: 201096.6250
Epoch 88

1/1 [==============================] - 0s 17ms/step - loss: 123185.6484 - val_loss: 219456.3438
Epoch 949/10000
1/1 [==============================] - 0s 18ms/step - loss: 120259.6016 - val_loss: 259655.1875
Epoch 950/10000
1/1 [==============================] - 0s 17ms/step - loss: 123896.4141 - val_loss: 213954.3594
Epoch 951/10000
1/1 [==============================] - 0s 17ms/step - loss: 121159.8125 - val_loss: 238524.2031
Epoch 952/10000
1/1 [==============================] - 0s 17ms/step - loss: 112523.9453 - val_loss: 252718.7812
Epoch 953/10000
1/1 [==============================] - 0s 18ms/step - loss: 124637.9922 - val_loss: 272522.7500
Epoch 954/10000
1/1 [==============================] - 0s 18ms/step - loss: 124965.2109 - val_loss: 212430.7500
Epoch 955/10000
1/1 [==============================] - 0s 20ms/step - loss: 114928.5703 - val_loss: 243757.9688
Epoch 956/10000
1/1 [==============================] - 0s 19ms/step - loss: 121313.4453 - val_loss: 230862.4219
Epoch 95

1/1 [==============================] - 0s 17ms/step - loss: 123588.1016 - val_loss: 244480.0312
Epoch 1022/10000
1/1 [==============================] - 0s 17ms/step - loss: 122059.5234 - val_loss: 225557.9688
Epoch 1023/10000
1/1 [==============================] - 0s 17ms/step - loss: 112784.8125 - val_loss: 237428.0312
Epoch 1024/10000
1/1 [==============================] - 0s 16ms/step - loss: 122864.2578 - val_loss: 244415.8125
Epoch 1025/10000
1/1 [==============================] - 0s 16ms/step - loss: 118193.3516 - val_loss: 243664.9219
Epoch 1026/10000
1/1 [==============================] - 0s 16ms/step - loss: 127087.3438 - val_loss: 235029.1875
Epoch 1027/10000
1/1 [==============================] - 0s 16ms/step - loss: 122811.5703 - val_loss: 268174.2812
Epoch 1028/10000
1/1 [==============================] - 0s 16ms/step - loss: 128029.0234 - val_loss: 227742.1562
Epoch 1029/10000
1/1 [==============================] - 0s 16ms/step - loss: 113674.7422 - val_loss: 237982.4688


In [28]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 59ms/step
[3862.535](test_target) - [[3793.7932]](prediction) = 68.74178710937485


In [29]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [30]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.945479,-1.273847,-0.903379,-0.785203,-1.284368,-1.255052,-0.439210,-1.233495,-1.334114,1.148205,...,1.277766,1.308949,1.412346,-1.412669,1.305056,-1.410138,1.251494,1.242982,-1.380085,1.414193
1,-0.438058,0.104961,-0.490611,-0.626022,0.129545,0.063049,1.383787,0.017693,0.260725,0.140893,...,-0.114016,-0.190804,-0.643250,0.763554,-1.124355,0.612152,-0.055376,-0.037329,0.422604,-0.713648
2,1.383537,1.168887,1.393990,1.411224,1.154823,1.192003,-0.944578,1.215803,1.073389,-1.289098,...,-1.163750,-1.118144,-0.769096,0.649115,-0.180701,0.797986,-1.196117,-1.205654,0.957481,-0.700545


train_target:


,Bahia - Consumo de Cimento (t)
0,2825.491
1,3349.109
2,3615.893


test_input:


...

,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Desemprego,Bahia - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.416125,-1.086745,-0.129344,-0.207609,0.000408,0.737609,0.749286,0.972824,-0.47095,1.108405,...,0.964765,-1.482779,-2.260447,1.940009,0.112434,-1.032368,-0.187335,-0.1229,0.783824,-1.669544


test_target:


,Bahia - Consumo de Cimento (t)
12,3862.535


1/1 [==============================] - 0s 35ms/step
Error: 68.74178710937485




[3615.88818359375,
 3687.5654296875,
 3934.103515625,
 3988.583251953125,
 3582.423095703125,
 3528.89892578125,
 3063.25341796875,
 3149.243408203125,
 3245.745361328125,
 3793.793212890625]

In [31]:
display(targets)
display(predictions)

[3744.322,
 3932.905,
 4055.14,
 3732.551999999999,
 3424.967999999999,
 3063.3,
 2990.152,
 3245.585,
 3803.9,
 3862.535]

[3615.88818359375,
 3687.5654296875,
 3934.103515625,
 3988.583251953125,
 3582.423095703125,
 3528.89892578125,
 3063.25341796875,
 3149.243408203125,
 3245.745361328125,
 3793.793212890625]

In [32]:
mae = mean_absolute_error(predictions, targets)
mae

217.02345800781268

In [33]:
porcentage = mae/np.mean(targets)
porcentage

0.06052748154266498